In [5]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
%cd -q ..

In [10]:
import lcdb
import os
import json
import lcpfn 
import torch as th
from functools import partial
import numpy as np
from matplotlib import pyplot as plt

DEFINE PATH

In [8]:
PATH = ""

In [4]:
# path = "C:\\Users\\mjaic\\OneDrive\\Desktop\\Delft\\Year 1\\Q3\\Deep Learning\\deep-learning-project\\notebooks\data.json"

def readDatasetJson(path):
    f = open(path)

    dataset = json.load(f)
    # print(len(dataset))
    
    x = []
    y = []

    for v in dataset.values():
        x.append(v[0])
        y.append(v[1])
        
    x_true = th.Tensor(x)
    y_true = th.Tensor(y)
    
    return x_true, y_true

import os
def find_file(root_dir, target_filename):
    for root, dirs, files in os.walk(root_dir):
        if target_filename in files:
            return os.path.join(root, target_filename)
    
    # If the loop completes without returning, the file was not found
    return None

def find_data():
    return find_file(os.getcwd(), 'data.json')

def get_data():
    return readDatasetJson(find_data())

# function producing batches for PFN training
def get_batch(
    batch_size,
    seq_len,
    num_features,
    device="cpu",
    noisy_target=True,
    **_,
):
    assert num_features == 1

    x_data, y_data = get_data()
    
    x_data = x_data[:batch_size, :]
    y_data = y_data[:batch_size, :]

    y_data_noisy = y_data.clone()

    x_data = x_data.view((num_features, batch_size, seq_len)).transpose(2, 0).to(device)
    y_data = y_data.transpose(1, 0).to(device)
    y_data_noisy = y_data.clone()

    return x_data, y_data, y_data_noisy

In [9]:
result = lcpfn.train_lcpfn(get_batch_func=get_batch, 
                         seq_len=26,
                         emsize=256,
                         nlayers=12,
                         lr=0.001,
                         batch_size=1651,
                         epochs=8)

th.save(result[2].state_dict(), os.path.join(PATH, "model_1651.pth"))

Using cpu:0 device
init dist
Not using distributed
DataLoader.__dict__ {'num_steps': 100, 'get_batch_kwargs': {'batch_size': 1, 'eval_pos_seq_len_sampler': <function train.<locals>.eval_pos_seq_len_sampler at 0x000002C5D9C8DF70>, 'seq_len_maximum': 26, 'device': 'cpu:0', 'num_features': 1, 'hyperparameters': {}}, 'num_features': 1}
torch.Size([3302, 26])
Style definition: None
Using a Transformer with 6.46 M parameters
torch.Size([3302, 26])
None
tensor([[[0.0050]],

        [[0.0072]],

        [[0.0100]],

        [[0.0141]],

        [[0.0200]],

        [[0.0285]],

        [[0.0401]],

        [[0.0566]],

        [[0.0801]],

        [[0.1133]],

        [[0.1158]],

        [[0.1173]],

        [[0.1602]],

        [[0.1818]],

        [[0.2265]],

        [[0.3204]],

        [[0.3336]],

        [[0.4531]],

        [[0.4611]],

        [[0.4904]],

        [[0.5478]],

        [[0.5857]],

        [[0.6408]],

        [[0.7039]],

        [[0.8016]],

        [[0.8098]]])
ten

c:\Users\mjaic\miniconda3\envs\lcpfn\lib\site-packages\torch\autocast_mode.py:162: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


torch.Size([3302, 26])
None
tensor([[[0.0050]],

        [[0.0072]],

        [[0.0100]],

        [[0.0141]],

        [[0.0200]],

        [[0.0285]],

        [[0.0401]],

        [[0.0566]],

        [[0.0801]],

        [[0.1133]],

        [[0.1158]],

        [[0.1173]],

        [[0.1602]],

        [[0.1818]],

        [[0.2265]],

        [[0.3204]],

        [[0.3336]],

        [[0.4531]],

        [[0.4611]],

        [[0.4904]],

        [[0.5478]],

        [[0.5857]],

        [[0.6408]],

        [[0.7039]],

        [[0.8016]],

        [[0.8098]]])
tensor([[1.0000],
        [1.0000],
        [1.0000],
        [1.0000],
        [0.9998],
        [0.9974],
        [0.9951],
        [0.9898],
        [0.9848],
        [0.9800],
        [0.9799],
        [0.9798],
        [0.9786],
        [0.9781],
        [0.9771],
        [0.9752],
        [0.9752],
        [0.9750],
        [0.9751],
        [0.9751],
        [0.9752],
        [0.9753],
        [0.9754],
        [0.9

In [ ]:
result = lcpfn.train_lcpfn(get_batch_func=get_batch, 
                         seq_len=26,
                         emsize=256,
                         nlayers=12,
                         lr=0.001,
                         batch_size=1000,
                         epochs=10)

th.save(result[2].state_dict(), os.path.join(PATH, "model_256_12.pth"))

In [ ]:
result = lcpfn.train_lcpfn(get_batch_func=get_batch, 
                         seq_len=26,
                         emsize=256,
                         nlayers=10,
                         lr=0.001,
                         batch_size=1000,
                         epochs=10)

th.save(result[2].state_dict(), os.path.join(PATH, "model_256_10.pth"))

In [ ]:
result = lcpfn.train_lcpfn(get_batch_func=get_batch, 
                         seq_len=26,
                         emsize=256,
                         nlayers=18,
                         lr=0.001,
                         batch_size=1000,
                         epochs=10)

th.save(result[2].state_dict(), os.path.join(PATH, "model_256_8.pth"))

In [ ]:
result = lcpfn.train_lcpfn(get_batch_func=get_batch, 
                         seq_len=26,
                         emsize=128,
                         nlayers=12,
                         lr=0.001,
                         batch_size=1000,
                         epochs=10)

th.save(result[2].state_dict(), os.path.join(PATH, "model_128_12.pth"))

In [ ]:
result = lcpfn.train_lcpfn(get_batch_func=get_batch, 
                         seq_len=26,
                         emsize=128,
                         nlayers=10,
                         lr=0.001,
                         batch_size=1000,
                         epochs=10)

th.save(result[2].state_dict(), os.path.join(PATH, "model_128_10.pth"))

In [ ]:
result = lcpfn.train_lcpfn(get_batch_func=get_batch, 
                         seq_len=26,
                         emsize=128,
                         nlayers=8,
                         lr=0.001,
                         batch_size=1000,
                         epochs=10)

th.save(result[2].state_dict(), os.path.join(PATH, "model_128_8.pth"))